In [1]:
import keras
import numpy as np
import tensorflow as tf

from absl import flags
from pathlib import Path
from operator import itemgetter
from keras.optimizers import Adam
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score

Using TensorFlow backend.


In [2]:
import importlib

import model as nn
importlib.reload(nn)

elsa_architecture = nn.elsa_architecture

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

lang = "en"
batch_size = 250
lr = 1e-3
epochs = 100
patience = 3
data_dir = "/data/elsa2"
checkpoint_dir = "./ckpt"
optimizer = "adam"

In [4]:
lstm_hidden = 512
lstm_drop = 0.5
final_drop = 0.5
embed_drop = 0.0
highway = False
compute_class_weight = False
multilabel = True

In [5]:
data_dir = Path(data_dir)
wv_path = (data_dir / "{:s}_wv.npy".format(lang)).__str__()
X_path = (data_dir / "{:s}_X.npy".format(lang)).__str__()
y_path = (data_dir / "{:s}_y.npy".format(lang)).__str__()
emoji_path = (data_dir / "{:s}_emoji.txt".format(lang)).__str__()

wv = np.load(wv_path, allow_pickle=True)
input_vec = np.load(X_path, allow_pickle=True)
input_label = np.load(y_path, allow_pickle=True)

nb_tokens = len(wv)
embed_dim = wv.shape[1]
input_len = len(input_label)
nb_classes = input_label.shape[1]
maxlen = input_vec.shape[1]

train_end = int(input_len*0.7)
val_end = int(input_len*0.9)

(X_train, y_train) = (input_vec[:train_end], input_label[:train_end])
(X_val, y_val) = (input_vec[train_end:val_end], input_label[train_end:val_end])
(X_test, y_test) = (input_vec[val_end:], input_label[val_end:])

if multilabel:
    def to_multilabel(y):
        outputs = []
        for i in range(nb_classes):
            outputs.append(y[:, i])
        return outputs

    y_train = to_multilabel(y_train)
    y_val = to_multilabel(y_val)
    y_test = to_multilabel(y_test)

model = elsa_architecture(nb_classes=nb_classes,
                          nb_tokens=nb_tokens,
                          maxlen=maxlen,
                          final_dropout_rate=final_drop,
                          embed_dropout_rate=embed_drop,
                          load_embedding=True,
                          pre_embedding=wv,
                          high=highway,
                          embed_dim=embed_dim,
                          multilabel=multilabel)
model.summary()

computed_class_weight = None

if multilabel:
    loss = "binary_crossentropy"
else:
    loss = "categorical_crossentropy"
    if compute_class_weight:
        y_train_sps = []
        for row in y_train:
            y_train_sps.extend(np.where(row)[0].tolist())
        computed_class_weight = class_weight.compute_class_weight(
            'balanced', list(range(nb_classes)), y_train_sps)
        print("computed class weight = {:s}".format(str(computed_class_weight)))

if optimizer == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optimizer == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

checkpoint_dir = Path(checkpoint_dir)
if not checkpoint_dir.exists():
    checkpoint_dir.mkdir()
checkpoint_weight_path = (checkpoint_dir / "elsa_{:s}.hdf5".format(lang)).__str__()

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto'),
    keras.callbacks.ModelCheckpoint(checkpoint_weight_path, monitor='val_loss',
                                    verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
]
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, y_val),
          class_weight=computed_class_weight,
          callbacks=callbacks,
          verbose=1)

freq = {line.split()[0]: int(line.split()[1]) for line in open(emoji_path).readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

if multilabel:
    y_pred = model.predict([X_test], batch_size=batch_size)
    y_pred = [np.squeeze(p) for p in y_pred]

    y_test_1d = np.array(y_test).flatten()
    y_pred_1d = np.array(y_pred).flatten()
    print(f1_score(y_test_1d, y_pred_1d > 0.5))
    print(classification_report(y_test_1d, y_pred_1d > 0.5))

    gold, pred = [], []
    for i in range(len(X_test)):
        each_gold, each_pred = [], []
        for c in range(nb_classes):
            if y_test[c][i] == 1.0:
                each_gold.append(c+1)
            else:
                each_gold.append(0)
            if y_pred[c][i] > 0.5:
                each_pred.append(c+1)
            else:
                each_pred.append(0)
        gold.extend(each_gold)
        pred.extend(each_pred)

    target_name = [""] + [e[0] for e in freq_topn]
    print(classification_report(gold, pred, target_names=target_name))
else:
    _, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(acc)

    y_pred = model.predict(X_test)
    print(classification_report(y_test.argmax(axis=1), y_pred.argmax(
        axis=1), target_names=[e[0] for e in freq_topn]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 200)      64132000    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 20, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2106921 samples, validate on 601978 samples
Epoch 1/100
2106921/2106921 [==============================] - 2760s 1ms/step - loss: 3.9323 - sigmoid_0_loss: 0.3936 - sigmoid_1_loss: 0.2572 - sigmoid_2_loss: 0.1816 - sigmoid_3_loss: 0.1491 - sigmoid_4_loss: 0.1444 - sigmoid_5_loss: 0.1148 - sigmoid_6_loss: 0.0705 - sigmoid_7_loss: 0.0734 - sigmoid_8_loss: 0.0134 - sigmoid_9_loss: 0.0746 - sigmoid_10_loss: 0.0622 - sigmoid_11_loss: 0.0913 - sigmoid_12_loss: 0.0106 - sigmoid_13_loss: 0.0436 - sigmoid_14_loss: 0.0734 - sigmoid_15_loss: 0.0859 - sigmoid_16_loss: 0.0391 - sigmoid_17_loss: 0.0589 - sigmoid_18_loss: 0.0428 - sigmoid_19_loss: 0.0635 - sigmoid_20_loss: 0.0813 - sigmoid_21_loss: 0.0712 - sigmoid_22_loss: 0.0469 - sigmoid_23_loss: 0.0581 - sigmoid_24_loss: 0.0762 - sigmoid_25_loss: 0.0696 - sigmoid_26_loss: 0.0466 - sigmoid_27_loss: 0.0630 - sigmoid_

    500/2106921 [..............................] - ETA: 45:04 - loss: 3.6626 - sigmoid_0_loss: 0.4205 - sigmoid_1_loss: 0.2267 - sigmoid_2_loss: 0.2057 - sigmoid_3_loss: 0.1360 - sigmoid_4_loss: 0.1226 - sigmoid_5_loss: 0.1093 - sigmoid_6_loss: 0.0675 - sigmoid_7_loss: 0.0457 - sigmoid_8_loss: 0.0105 - sigmoid_9_loss: 0.0746 - sigmoid_10_loss: 0.0357 - sigmoid_11_loss: 0.1113 - sigmoid_12_loss: 0.0104 - sigmoid_13_loss: 0.0227 - sigmoid_14_loss: 0.0573 - sigmoid_15_loss: 0.0647 - sigmoid_16_loss: 0.0289 - sigmoid_17_loss: 0.0306 - sigmoid_18_loss: 0.0381 - sigmoid_19_loss: 0.0695 - sigmoid_20_loss: 0.0549 - sigmoid_21_loss: 0.0913 - sigmoid_22_loss: 0.0454 - sigmoid_23_loss: 0.0750 - sigmoid_24_loss: 0.0637 - sigmoid_25_loss: 0.0965 - sigmoid_26_loss: 0.0358 - sigmoid_27_loss: 0.0570 - sigmoid_28_loss: 0.0272 - sigmoid_29_loss: 0.0526 - sigmoid_30_loss: 0.0607 - sigmoid_31_loss: 0.0395 - sigmoid_32_loss: 0.0223 - sigmoid_33_loss: 0.0199 - sigmoid_34_loss: 0.0533 - sigmoid_35_loss: 0.04

2106921/2106921 [==============================] - 2599s 1ms/step - loss: 3.7660 - sigmoid_0_loss: 0.3808 - sigmoid_1_loss: 0.2497 - sigmoid_2_loss: 0.1754 - sigmoid_3_loss: 0.1458 - sigmoid_4_loss: 0.1386 - sigmoid_5_loss: 0.1096 - sigmoid_6_loss: 0.0665 - sigmoid_7_loss: 0.0673 - sigmoid_8_loss: 0.0111 - sigmoid_9_loss: 0.0725 - sigmoid_10_loss: 0.0570 - sigmoid_11_loss: 0.0881 - sigmoid_12_loss: 0.0089 - sigmoid_13_loss: 0.0402 - sigmoid_14_loss: 0.0711 - sigmoid_15_loss: 0.0829 - sigmoid_16_loss: 0.0347 - sigmoid_17_loss: 0.0563 - sigmoid_18_loss: 0.0377 - sigmoid_19_loss: 0.0593 - sigmoid_20_loss: 0.0780 - sigmoid_21_loss: 0.0664 - sigmoid_22_loss: 0.0454 - sigmoid_23_loss: 0.0561 - sigmoid_24_loss: 0.0728 - sigmoid_25_loss: 0.0659 - sigmoid_26_loss: 0.0443 - sigmoid_27_loss: 0.0611 - sigmoid_28_loss: 0.0432 - sigmoid_29_loss: 0.0381 - sigmoid_30_loss: 0.0462 - sigmoid_31_loss: 0.0436 - sigmoid_32_loss: 0.0340 - sigmoid_33_loss: 0.0211 - sigmoid_34_loss: 0.0529 - sigmoid_35_loss: 

    500/2106921 [..............................] - ETA: 43:22 - loss: 3.6641 - sigmoid_0_loss: 0.3648 - sigmoid_1_loss: 0.2574 - sigmoid_2_loss: 0.1456 - sigmoid_3_loss: 0.1665 - sigmoid_4_loss: 0.1102 - sigmoid_5_loss: 0.1064 - sigmoid_6_loss: 0.0409 - sigmoid_7_loss: 0.0346 - sigmoid_8_loss: 0.0036 - sigmoid_9_loss: 0.0722 - sigmoid_10_loss: 0.0798 - sigmoid_11_loss: 0.0585 - sigmoid_12_loss: 0.0032 - sigmoid_13_loss: 0.0542 - sigmoid_14_loss: 0.0445 - sigmoid_15_loss: 0.1023 - sigmoid_16_loss: 0.0330 - sigmoid_17_loss: 0.0659 - sigmoid_18_loss: 0.0339 - sigmoid_19_loss: 0.0915 - sigmoid_20_loss: 0.0768 - sigmoid_21_loss: 0.0541 - sigmoid_22_loss: 0.0622 - sigmoid_23_loss: 0.0675 - sigmoid_24_loss: 0.0411 - sigmoid_25_loss: 0.0468 - sigmoid_26_loss: 0.0254 - sigmoid_27_loss: 0.1010 - sigmoid_28_loss: 0.0207 - sigmoid_29_loss: 0.0160 - sigmoid_30_loss: 0.0373 - sigmoid_31_loss: 0.0440 - sigmoid_32_loss: 0.0701 - sigmoid_33_loss: 0.0095 - sigmoid_34_loss: 0.0410 - sigmoid_35_loss: 0.09

...

2106921/2106921 [==============================] - 2545s 1ms/step - loss: 3.6369 - sigmoid_0_loss: 0.3669 - sigmoid_1_loss: 0.2419 - sigmoid_2_loss: 0.1689 - sigmoid_3_loss: 0.1419 - sigmoid_4_loss: 0.1328 - sigmoid_5_loss: 0.1053 - sigmoid_6_loss: 0.0633 - sigmoid_7_loss: 0.0619 - sigmoid_8_loss: 0.0104 - sigmoid_9_loss: 0.0702 - sigmoid_10_loss: 0.0531 - sigmoid_11_loss: 0.0854 - sigmoid_12_loss: 0.0083 - sigmoid_13_loss: 0.0376 - sigmoid_14_loss: 0.0690 - sigmoid_15_loss: 0.0798 - sigmoid_16_loss: 0.0317 - sigmoid_17_loss: 0.0536 - sigmoid_18_loss: 0.0339 - sigmoid_19_loss: 0.0554 - sigmoid_20_loss: 0.0745 - sigmoid_21_loss: 0.0618 - sigmoid_22_loss: 0.0437 - sigmoid_23_loss: 0.0542 - sigmoid_24_loss: 0.0696 - sigmoid_25_loss: 0.0621 - sigmoid_26_loss: 0.0419 - sigmoid_27_loss: 0.0590 - sigmoid_28_loss: 0.0403 - sigmoid_29_loss: 0.0360 - sigmoid_30_loss: 0.0430 - sigmoid_31_loss: 0.0416 - sigmoid_32_loss: 0.0330 - sigmoid_33_loss: 0.0193 - sigmoid_34_loss: 0.0510 - sigmoid_35_loss: 

    500/2106921 [..............................] - ETA: 43:18 - loss: 3.5018 - sigmoid_0_loss: 0.3170 - sigmoid_1_loss: 0.2197 - sigmoid_2_loss: 0.1623 - sigmoid_3_loss: 0.0946 - sigmoid_4_loss: 0.1473 - sigmoid_5_loss: 0.1100 - sigmoid_6_loss: 0.0465 - sigmoid_7_loss: 0.0553 - sigmoid_8_loss: 0.0228 - sigmoid_9_loss: 0.0650 - sigmoid_10_loss: 0.0549 - sigmoid_11_loss: 0.1300 - sigmoid_12_loss: 0.0118 - sigmoid_13_loss: 0.0539 - sigmoid_14_loss: 0.0679 - sigmoid_15_loss: 0.0863 - sigmoid_16_loss: 0.0168 - sigmoid_17_loss: 0.0401 - sigmoid_18_loss: 0.0104 - sigmoid_19_loss: 0.0919 - sigmoid_20_loss: 0.0690 - sigmoid_21_loss: 0.0526 - sigmoid_22_loss: 0.0150 - sigmoid_23_loss: 0.0403 - sigmoid_24_loss: 0.0572 - sigmoid_25_loss: 0.0465 - sigmoid_26_loss: 0.0419 - sigmoid_27_loss: 0.0380 - sigmoid_28_loss: 0.0171 - sigmoid_29_loss: 0.0409 - sigmoid_30_loss: 0.0310 - sigmoid_31_loss: 0.0534 - sigmoid_32_loss: 0.0410 - sigmoid_33_loss: 0.0151 - sigmoid_34_loss: 0.0426 - sigmoid_35_loss: 0.04

2106921/2106921 [==============================] - 2543s 1ms/step - loss: 3.5972 - sigmoid_0_loss: 0.3624 - sigmoid_1_loss: 0.2394 - sigmoid_2_loss: 0.1667 - sigmoid_3_loss: 0.1406 - sigmoid_4_loss: 0.1310 - sigmoid_5_loss: 0.1044 - sigmoid_6_loss: 0.0625 - sigmoid_7_loss: 0.0605 - sigmoid_8_loss: 0.0101 - sigmoid_9_loss: 0.0694 - sigmoid_10_loss: 0.0522 - sigmoid_11_loss: 0.0846 - sigmoid_12_loss: 0.0081 - sigmoid_13_loss: 0.0368 - sigmoid_14_loss: 0.0681 - sigmoid_15_loss: 0.0788 - sigmoid_16_loss: 0.0308 - sigmoid_17_loss: 0.0531 - sigmoid_18_loss: 0.0330 - sigmoid_19_loss: 0.0544 - sigmoid_20_loss: 0.0735 - sigmoid_21_loss: 0.0606 - sigmoid_22_loss: 0.0431 - sigmoid_23_loss: 0.0536 - sigmoid_24_loss: 0.0689 - sigmoid_25_loss: 0.0612 - sigmoid_26_loss: 0.0412 - sigmoid_27_loss: 0.0581 - sigmoid_28_loss: 0.0396 - sigmoid_29_loss: 0.0353 - sigmoid_30_loss: 0.0422 - sigmoid_31_loss: 0.0408 - sigmoid_32_loss: 0.0327 - sigmoid_33_loss: 0.0189 - sigmoid_34_loss: 0.0503 - sigmoid_35_loss: 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

                   0.99      1.00      0.99  18962307
           😂       0.56      0.11      0.19     53014
           😭       0.52      0.03      0.05     25023
           ❤       0.71      0.02      0.04     16602
           🤣       0.29      0.00      0.00     11424
           😍       0.64      0.01      0.02     11829
           🥺       0.54      0.02      0.05      9305
           💜       0.63      0.10      0.18      5267
           🔥       0.66      0.14      0.22      5587
           ♀       0.63      0.66      0.65      1916
           💕       0.74      0.00      0.01      4927
           🙏       0.66      0.24      0.35      5626
           😩       0.46      0.00      0.00      6178
           ♂       0.57      0.27      0.37      1169
           👏       0.61      0.12      0.20      3192
           🥰       0.00      0.00      0.00      4646
           😊       0.74      0.01      0.01      5827
           🤦       0.63    